## Data collection and imputation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

football = Fantasy()

This is all cleaning the data

In [3]:
df = pd.read_excel('2022-offense.xlsx', sheet_name='Offense_Prior_Actuals', header=3)

# Drop columns that we don't need
drop_columns = ['GS', 'Bye', 'Notes', 'Rank', 'Y! Roto', 'Δ', 'Y! Old', 'Std', 'Δ.1', 'Std Old', 'PPR', 'Δ.2', 'PPR Old', '% Own', 'Fan Pts', 'PPG']
df = df.drop(drop_columns, axis=1)

# Change names of duplicate columns
updated_columns = {'Yds': 'Passing_Yds', 'Yds.1': 'Rushing_Yds', 'Yds.2': 'Receiving_Yds', 'Yds.3': 'Return_Yds', 
                   'TD': 'Passing_Td', 'TD.1': 'Rushing_Td', 'TD.2': 'Receiving_Td', 'TD.3': 'Return_Td', 
                   '1st': 'Passing_1st', '1st.1': 'Rushing_1st', '1st.2': 'Receiving_1st'}
df = df.rename(columns=updated_columns)
df


,Player,Pos,Team,GP,Comp,Inc,Passing_Yds,Passing_Td,Int,Pic6,...,Tgt,Rec,Receiving_Yds,Receiving_Td,Receiving_1st,Return_Yds,Return_Td,2PT,Total,Lost
0,J. Allen,QB,Buf,17,409,237,4407,36,15,0,...,0,0,0,0,0,0,0,3,8,3
1,J. Herbert,QB,LAC,17,443,229,5014,38,15,3,...,0,0,0,0,0,0,0,6,1,1
2,P. Mahomes,QB,KC,17,436,222,4839,37,13,1,...,0,0,0,0,0,0,0,2,9,4
3,T. Brady,QB,TB,17,485,234,5316,43,12,2,...,0,0,0,0,0,0,0,0,4,3
4,J. Burrow,QB,Cin,16,366,154,4611,34,14,2,...,0,0,0,0,0,0,0,1,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,E. St. Brown,WR,Chi,13,0,0,0,0,0,0,...,17,9,98,0,6,17,0,0,0,0
371,P. Taylor,RB,GB,9,0,0,0,0,0,0,...,2,2,3,0,1,0,0,0,0,0
372,K. Smith,RB,Atl,17,0,0,0,0,0,0,...,10,9,56,0,3,26,0,0,0,0
373,G. Gilbert,QB,LV,1,20,11,194,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

passing_advanced = football.getAdvancedStats("passing", "2021")

print("Rushing")
rushing_advanced = football.getAdvancedStats("rushing", "2021")
#url = "https://www.pro-football-reference.com/years/2021/rushing_advanced.htm"
#temp = pd.read_html(url)

# temp


https://www.pro-football-reference.com/years/2021/passing_advanced.htm
Rushing
https://www.pro-football-reference.com/years/2021/rushing_advanced.htm


ValueError: No tables found

In [48]:

# For the rushing, we need to use selenium to account for the dynamically implemented dataset

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.pro-football-reference.com/years/2021/rushing_advanced.htm'

# Set up ChromeDriver
driver = webdriver.Chrome()
driver.get(url)

# Wait for the page to load
driver.implicitly_wait(5)

# Get the page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table element by id
table = soup.find('table', {'id': 'advanced_rushing'})

# Quit the browser
driver.quit()

In [49]:
headers = table.find_all('tr')[1].find_all('th') # Find all headers in HTML
players = table.find_all('tbody')[0] # Find all players (which will be our rows)


# Get column headers
cols = []
for th in headers:
    cols.append(th.text.strip())
cols = cols[1:] # Dont want to include the row number

# Get table rows
rows = []

# Loop through each 'tr' tag
for tr in players.find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    if row:
        rows.append(row)

# Make pandas df
df = pd.DataFrame(rows, columns=cols)




We are not using the player contracts yet but here is where we scrape that data

In [ ]:
player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Jay Cutler Quarterback | 2014-2020 (FA: 2021)...,30,7,"$126,700,000","$18,100,000",NaN,"$38,000,000","$54,000,000"
1,2,Cam Newton Quarterback | 2015-2020 (FA: 2021),26,5,"$103,800,000","$20,760,000","$22,500,000","$41,000,000","$60,000,000"
2,3,Andy Dalton Quarterback | 2014-2020 (FA: 2021...,26,6,"$96,000,000","$16,000,000","$12,000,000","$17,000,000","$17,000,000"
3,4,Kirk Cousins Quarterback | 2018-2020 (FA: 2021),29,3,"$84,000,000","$28,000,000","$3,000,000","$84,000,000","$84,000,000"
4,5,Ryan Tannehill Quarterback | 2015-2020 (FA: 2...,26,4,"$77,000,000","$19,250,000","$11,500,000","$21,500,000","$45,000,000"
...,...,...,...,...,...,...,...,...,...
95,96,Paxton Lynch Quarterback | 2016-2020 (FA: 202...,22,4,"$9,476,296","$2,369,074","$5,091,852","$7,884,074","$7,884,074"
96,97,Lee Smith Tight End | 2018-2020 (FA: 2021) O...,30,3,"$9,000,000","$3,000,000",NaN,"$3,000,000","$3,000,000"
97,97,Sammy Watkins Wide Receiver | 2020-2020 (FA: ...,26,1,"$9,000,000","$9,000,000","$1,000,000","$8,000,000","$8,000,000"
98,97,David Andrews Center | 2017-2020 (FA: 2021),24,3,"$9,000,000","$3,000,000","$1,600,000","$1,600,000","$1,600,000"


Here is where we collect isolate the data for quarterbacks and merge the basic dataframe with one of the advanced statistics ones.

In [ ]:
QB_data, RB_data, WR_data, TE_data = football.makeRegularData(df)

In [ ]:
qb_merged = football.addAdvanced(QB_data, passing_advanced)
qb_merged

,Player_x,Pos_x,Team,GP,Comp,Inc,Passing_Yds,Passing_Td,Int,Pic6,...,Cmp,Att_y,Yds,IAY,IAY/PA,CAY,CAY/Cmp,CAY/PA,YAC,YAC/Cmp
0,J. Allen,QB,Buf,17,409,237,4407,36,15,0,...,409,646,4407,5304,8.2,2664,6.5,4.1,1743,4.3
1,B. Allen,QB,Cin,6,17,17,149,2,0,0,...,409,646,4407,5304,8.2,2664,6.5,4.1,1743,4.3
2,J. Herbert,QB,LAC,17,443,229,5014,38,15,3,...,443,672,5014,5076,7.6,2627,5.9,3.9,2387,5.4
3,P. Mahomes,QB,KC,17,436,222,4839,37,13,1,...,436,658,4839,4829,7.3,2140,4.9,3.3,2699,6.2
4,T. Brady,QB,TB,17,485,234,5316,43,12,2,...,485,719,5316,5791,8.1,2762,5.7,3.8,2554,5.3
5,J. Burrow,QB,Cin,16,366,154,4611,34,14,2,...,366,520,4611,4192,8.1,2332,6.4,4.5,2279,6.2
6,M. Stafford,QB,LAR,17,404,197,4886,41,17,4,...,404,601,4886,5092,8.5,2760,6.8,4.6,2126,5.3
7,D. Prescott,QB,Dal,16,410,186,4449,37,10,1,...,410,596,4449,4592,7.7,2425,5.9,4.1,2024,4.9
8,A. Rodgers,QB,GB,16,366,165,4115,37,4,0,...,366,531,4115,4099,7.7,1947,5.3,3.7,2168,5.9
9,R. Tannehill,QB,Ten,17,357,174,3734,21,14,0,...,357,531,3734,3959,7.5,1958,5.5,3.7,1776,5.0


In [ ]:
rb_merged = football.addAdvanced(RB_data, rushing_advanced)
# rb_merged

NameError: name 'rushing_advanced' is not defined